# IndicF5 Gradio Demo

This notebook requires a **GPU runtime**. Go to **Runtime → Change runtime type → T4 GPU** before running.

In [ ]:
# Check GPU availability - THIS MODEL REQUIRES GPU
import torch
if not torch.cuda.is_available():
    raise RuntimeError('❌ GPU not available! Go to Runtime → Change runtime type → T4 GPU')
print(f'✅ GPU available: {torch.cuda.get_device_name(0)}')

In [ ]:
# Clear ALL caches for fresh start
!rm -rf ~/.cache/huggingface/modules/
!rm -rf ~/.cache/huggingface/hub/models--ai4bharat--IndicF5/
print('✅ Caches cleared')

In [ ]:
# Install IndicF5 directly (includes all deps)
%pip install -q git+https://github.com/ai4bharat/IndicF5.git
%pip install -q gradio

In [ ]:
# Use the official IndicF5 inference API (not HuggingFace AutoModel)
import torch
import gradio as gr
import tempfile
import soundfile as sf
import numpy as np

from indicf5 import IndicF5

print('Loading IndicF5 model...')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = IndicF5(device=device)
print(f'✅ Model loaded on {device}')

def synthesize_speech(text, ref_audio, ref_text):
    if not text or ref_audio is None or not ref_text:
        return 'Error: Please provide text, reference audio, and reference text.'
    
    # Get audio from Gradio
    if isinstance(ref_audio, tuple):
        sample_rate, audio_data = ref_audio
    else:
        return 'Error: Invalid audio format'
    
    # Save temp file
    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp:
        sf.write(tmp.name, audio_data, sample_rate)
        
        # Generate speech using official API
        audio_out = model.generate(text, ref_audio_path=tmp.name, ref_text=ref_text)
    
    # Return audio
    if hasattr(audio_out, 'cpu'):
        audio_out = audio_out.cpu().numpy()
    
    return 24000, audio_out

# Create Gradio interface
with gr.Blocks(title='IndicF5 TTS') as iface:
    gr.Markdown('# IndicF5 Text-to-Speech')
    
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label='Text to synthesize', lines=3)
            ref_audio = gr.Audio(label='Reference Audio', type='numpy')
            ref_txt = gr.Textbox(label='Reference Text (transcription of audio)')
            btn = gr.Button('Generate Speech', variant='primary')
        with gr.Column():
            out = gr.Audio(label='Generated Audio')
    
    btn.click(synthesize_speech, inputs=[txt, ref_audio, ref_txt], outputs=[out])

iface.launch(share=True, debug=True)